In [13]:
!pip3 install git+https://github.com/Centre-automatique-et-systemes/learn_observe_KKL.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null


In [1]:
import sys ; sys.path.append('../')
import torch.optim as optim
import torch
import seaborn as sb
import pytorch_lightning as pl
import numpy as np

from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from learn_KKL.luenberger_observer import LuenbergerObserver
from learn_KKL.system import RevDuffing
from learn_KKL.learner import Learner
from learn_KKL.utils import generate_mesh

sb.set_style('whitegrid')

In [2]:
# Generate the data
system = RevDuffing()
data = generate_mesh(np.array([[-1., 1.], [-1., 1.]]), 10000, method='LHS')
data, val_data = train_test_split(data, test_size=0.3, shuffle=True)

# Create the observer (autoencoder design
# D = torch.tensor([[-1.1832, -0.0000, -0.0000],
#                   [-0.0000, -0.9370,  0.8939],
#                   [-0.0000, -0.8939, -0.9370]])
# R = torch.tensor([[np.cos(0.8), - np.sin(0.8), 0],
#                   [np.sin(0.8), np.cos(0.8), 0],
#                   [0., 0., 1]])
# D = torch.matmul(torch.matmul(R, D), R.t())  # rotated D
observer = LuenbergerObserver(dim_x=2, dim_y=1, method="Autoencoder", wc=0.2,
                              recon_lambda=0.8)
observer.set_dynamics(system)

# Train using pytorch-lightning and the learner class
# Options for training
trainer_options={'max_epochs': 15}
optimizer_options = {'weight_decay': 1e-6}
scheduler_options = {'mode': 'min', 'factor': 0.1, 'patience': 3,
                     'threshold': 5e-4, 'verbose': True}
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', min_delta=5e-4, patience=3, verbose=False, mode='min')
# Instantiate learner
learner = Learner(observer=observer, system=system, training_data=data,
                  validation_data=val_data, method='Autoencoder',
                  batch_size=10, lr=5e-4, optimizer=optim.Adam,
                  optimizer_options=optimizer_options,
                  scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                  scheduler_options=scheduler_options)
# Define logger and checkpointing
logger = TensorBoardLogger(save_dir=learner.results_folder + '/tb_logs')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
trainer = pl.Trainer(
    callbacks=[stopper, checkpoint_callback], **trainer_options, logger=logger,
    log_every_n_steps=1, check_val_every_n_epoch=3)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Results saved in in /Users/mona/PhD_code/learn_observe_KKL/src/jupyter_notebooks/runs/RevDuffing/Autoencoder/exp_3


In [3]:
# To see logger in tensorboard, copy the following output name_of_folder
print(f'Logs stored in {learner.results_folder}/tb_logs')
# which should be similar to jupyter_notebooks/runs/method/exp_0/tb_logs/
# Then type this in terminal:
# tensorboard --logdir=name_of_folder --port=8080

# Train and save results
trainer.fit(learner)
learner.save_results(limits=np.array([[-1, 1.], [-1., 1.]]), nb_trajs=10,
                     tsim=(0, 60), dt=1e-2,
                     checkpoint_path=checkpoint_callback.best_model_path)


  | Name  | Type               | Params
---------------------------------------------
0 | model | LuenbergerObserver | 26.1 K
---------------------------------------------
26.1 K    Trainable params
0         Non-trainable params
26.1 K    Total params
0.104     Total estimated model params size (MB)


Logs stored in /Users/mona/PhD_code/learn_observe_KKL/src/jupyter_notebooks/runs/RevDuffing/Autoencoder/exp_3/tb_logs


Validation sanity check: 0it [00:00, ?it/s]

/Users/mona/PhD_code/learn_observe_KKL/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:372: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/mona/PhD_code/learn_observe_KKL/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/mona/PhD_code/learn_observe_KKL/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in t

Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Saved model in /Users/mona/PhD_code/learn_observe_KKL/src/jupyter_notebooks/runs/RevDuffing/Autoencoder/exp_3


No handles with labels found to put in legend.


Shape of mesh for evaluation: torch.Size([10000, 5])


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
